In [ ]:
# 1. 环境设置
import os
import sys
import torch
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# 添加项目根目录到Python路径
project_root = Path('.').absolute()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# 导入完备的应用框架
from style_trajectory_app import StyleTrajectoryApp

# 检查CUDA可用性
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 设置路径 - 请修改为你的实际路径
CHECKPOINT_PATH = "/path/to/your/diffusiondrive_style_checkpoint.ckpt"
DATASET_PATH = "/path/to/your/navsim/dataset"

print(f"检查点路径: {CHECKPOINT_PATH}")
print(f"数据集路径: {DATASET_PATH}")
print("注意：请将上面的路径修改为你的实际路径")


In [ ]:
# 2. 应用初始化 - 一步到位！
print("正在初始化StyleTrajectoryApp...")
print("这包括模型加载、数据管理器初始化等，可能需要一些时间...")

try:
    # 一行代码初始化完整应用
    app = StyleTrajectoryApp(
        checkpoint_path=CHECKPOINT_PATH,
        dataset_path=DATASET_PATH,
        lr=6e-4
    )
    
    print("🎉 应用初始化成功!")
    print("\n应用信息:")
    print(app)
    
    print("\n可用的驾驶风格:")
    for style in app.get_available_styles():
        info = app.get_style_info(style)
        print(f"  - {style.capitalize()}: {info['description']}")
        
except Exception as e:
    print(f"❌ 应用初始化失败: {e}")
    print("请检查路径设置是否正确")
    import traceback
    traceback.print_exc()


In [ ]:
# 3. 随机场景选择
print("正在随机选择一个测试场景...")

try:
    # 自动随机选择场景 - 无需手动指定token！
    scene, scene_token = app.get_random_scene()
    
    print("✅ 随机场景选择成功!")
    print(f"选中场景: {scene_token}")
    
    # 获取场景详细信息
    scene_info = app.data_manager.get_scene_info(scene_token)
    print(f"\n场景信息:")
    print(f"  - Token: {scene_info['token']}")
    print(f"  - 地图: {scene_info.get('map_name', '未知')}")
    print(f"  - 日志: {scene_info.get('log_name', '未知')}")
    print(f"  - 相机帧数: {scene_info.get('camera_frames', 0)}")
    print(f"  - LiDAR帧数: {scene_info.get('lidar_frames', 0)}")
    print(f"  - 自车状态帧数: {scene_info.get('ego_status_frames', 0)}")
    
except Exception as e:
    print(f"❌ 场景选择失败: {e}")
    print("可能是数据集路径问题，请检查DATASET_PATH设置")
    import traceback
    traceback.print_exc()


In [ ]:
# 4. 一键风格推理 + 可视化
print("正在进行完整的风格演示...")
print("包括：三种风格推理 + 轨迹对比可视化")
print("这可能需要几秒钟时间，请耐心等待...")

try:
    # 一行代码完成所有工作：推理 + 可视化
    demo_result = app.run_style_demo(scene_token=scene_token)
    
    print("🎉 风格演示完成!")
    print(f"\n演示结果:")
    print(f"  - 场景: {demo_result['scene_token'][:12]}...")
    print(f"  - 地图: {demo_result['scene_metadata'].get('map_name', '未知')}")
    print(f"  - 推理时间: {demo_result['prediction_time']:.2f}秒")
    print(f"  - 总演示时间: {demo_result['demo_time']:.2f}秒")
    
    print(f"\n生成的轨迹风格:")
    for style_name in demo_result['trajectories'].keys():
        trajectory = demo_result['trajectories'][style_name]
        print(f"  - {style_name.capitalize()}: {trajectory.shape}")
        
        # 显示轨迹基本统计
        traj_np = trajectory.detach().cpu().numpy()[0]
        x_coords = traj_np[:, 0]
        y_coords = traj_np[:, 1]
        distances = np.sqrt(np.diff(x_coords)**2 + np.diff(y_coords)**2)
        total_distance = np.sum(distances)
        
        print(f"    起点: ({x_coords[0]:.2f}, {y_coords[0]:.2f})")
        print(f"    终点: ({x_coords[-1]:.2f}, {y_coords[-1]:.2f})")
        print(f"    总距离: {total_distance:.2f}米")
    
    # 显示可视化图表
    plt.show()
    
    print("\n✅ 风格对比图已显示！")
    print("🔴 红色 = Aggressive (激进)")
    print("🔵 蓝色 = Normal (正常)")  
    print("🟢 绿色 = Conservative (保守)")
    
except Exception as e:
    print(f"❌ 风格演示失败: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# 5. 尝试不同场景 (可选)
print("想尝试其他场景吗？运行下面的代码可以快速切换到新的随机场景：")
print()

# 提供快速切换场景的代码示例
print("# 快速演示新场景的代码：")
print("demo_result = app.run_style_demo()  # 自动选择新的随机场景")
print("plt.show()")
print()

# 显示一些有用的信息
print("📊 数据集统计信息:")
dataset_info = app.get_dataset_info()
print(f"  - 总场景数: {dataset_info['total_scenes']}")
print(f"  - 地图数量: {dataset_info['num_map_locations']}")
print(f"  - 日志数量: {dataset_info['num_logs']}")

print(f"\n🔧 模型信息:")
model_info = app.get_model_info()
print(f"  - 模型类型: {model_info['model_type']}")
print(f"  - 设备: {model_info['device']}")
print(f"  - 参数数量: {model_info['num_parameters']:,}")

print(f"\n🎯 可用操作:")
print("  - app.run_style_demo()  # 新随机场景演示")
print("  - app.get_random_scenes(5)  # 获取5个随机场景token")
print("  - app.predict_all_styles(scene_token)  # 指定场景推理")
print("  - app.visualize_style_comparison(trajectories)  # 手动可视化")
